In [5]:
import os
import shutil
import tempfile

os.environ["OMP_NUM_THREADS"] = "8"
import numpy as np

import hapod as hp

In [ ]:
work_dir = "temp"
os.makedirs(work_dir, exist_ok=True)

n_rows = 1000
n_cols = 1000
n_chunks = 7

X = hp.random_matrix(n_rows, n_cols, n_cols)

chunks_fnames = []
for i, x in enumerate(np.array_split(X, n_chunks, axis=1)):
    cols_fname = os.path.join(work_dir, f"data_{i:4d}.npy")
    np.save(cols_fname, x)

    chunks_fnames.append(cols_fname)

print(chunks_fnames)



shutil.rmtree("temp")
